In [1]:
%reload_ext autoreload
%autoreload 2

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from pathlib import Path

In [5]:
path = Path('/root/.fastai/meggitt')

In [6]:
path

PosixPath('/root/.fastai/meggitt')

In [7]:
df = pd.read_csv(path/'meggitt_prices.csv', sep=',')

In [8]:
df.columns = ['customer_region', 'customer_segment', 'customer_division', 'customer_planning_entity', 'customer_account_manager', 'product_type_of_sales', 
              'product_segment', 'product_planning_entity', 'product_description', 'product_production_site', 'product_platform',
             'year', 'month', 'planning_price']

In [9]:
df.head(7)

,customer_region,customer_segment,customer_division,customer_planning_entity,customer_account_manager,product_type_of_sales,product_segment,product_planning_entity,product_description,product_production_site,product_platform,year,month,planning_price
0,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,2,561.362002
1,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,3,561.362002
2,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,4,561.362002
3,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,6,561.362002
4,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,5,561.362002
5,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,7,561.362002
6,EMEA,TBD,Civil,TBD,Unknown,MRO,Fire Detection Simi,6360-13,450/170C-10.0M F/O DET,MCS US Simi,AW139,2019,10,561.362002


In [11]:
df.shape

(378263, 14)

In [13]:
df.describe()

,year,month,planning_price
count,378263.000000,378263.000000,378263.000000
mean,2019.500001,6.500015,4825.761828
std,0.500001,3.452050,14475.787886
min,2019.000000,1.000000,-118423.169231
25%,2019.000000,4.000000,385.763456
50%,2020.000000,7.000000,1453.050427
75%,2020.000000,9.500000,4484.804453
max,2020.000000,12.000000,314929.769231


In [14]:
df = pd.get_dummies(df)

In [16]:
df.head()

,year,month,planning_price,customer_region_APAC,customer_region_Americas,customer_region_EMEA,customer_region_Not Applicable,customer_region_Unknown,customer_segment_Aircraft Operators,customer_segment_Airframe OEM,...,"product_platform_UH60 Black Hawk (T70,Unknown",product_platform_Unknown,"product_platform_Unknown,V-22 Osprey","product_platform_Unknown,VALVE","product_platform_Unknown,WG13/Lynx (Airframe)",product_platform_V-22 Osprey,product_platform_V2500,product_platform_V2500-A1,product_platform_VALVE,product_platform_WG13/Lynx (Airframe)
0,2019,2,561.362002,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019,3,561.362002,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019,4,561.362002,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019,6,561.362002,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019,5,561.362002,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.shape

(378263, 6358)

In [22]:
dep_var = 'planning_price'
labels = np.array(df[dep_var])
features= df.drop(dep_var, axis = 1)

In [23]:
labels.shape

(378263,)

In [25]:
features.shape

(378263, 6357)

In [26]:
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [33]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [38]:
features.shape[0] * 0.8

302610.4

In [39]:
# Split the data into training and testing sets
split = 302610
train_features, test_features, train_labels, test_labels = features[:split], features[split:], labels[:split], labels[split:]

In [40]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (302610, 6357)
Training Labels Shape: (302610,)
Testing Features Shape: (75653, 6357)
Testing Labels Shape: (75653,)


In [41]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [ ]:
rf.fit(train_features, train_labels);